In [10]:
!pip install langchain openai --upgrade --quiet

<a href="https://colab.research.google.com/github/qxr777/llm-application-code/blob/master/function_call/function-call-based-on-openai-sdk.ipynb#scrollTo=9347fc9213a0b887" target="_parent">Open In Colab</a>

### 基于OpenAI的Python SDK使用函数调用

In [19]:
from dotenv import load_dotenv
load_dotenv()

# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

True

In [12]:
model = 'gpt-3.5-turbo-0613'

In [13]:
function_descriptions = [
  {
      "name": "get_student_score",
      "description": "Get the student score by given his or her name",
      "parameters": {
          "type": "object",
          "properties": {
              "name": {
                  "type": "string",
                  "description": "The student's name",
              }
          },
          "required": ["name"],
      },
    }
]

import json

SCORES = { 'Alex': 90, 'Lucy': 60 }
def get_student_score(name):

    """Get the student score by given his or her name"""

    score = {
        "name": name,
        "score": SCORES[name]
    }
    return json.dumps(score)

In [14]:
user_query = "What's the performance of Lucy in the school this year?"

In [15]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(model=model,
                                          messages=[{"role": "user", "content": user_query}],
                                          functions=function_descriptions,
                                          function_call="auto")
ai_response_message = response.choices[0].message
print(ai_response_message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "name": "Lucy"\n}', name='get_student_score'), tool_calls=None)


In [16]:
import inspect

def get_function_parameter_names(function):
    """
    获取指定函数的参数名列表。
    
    参数:
    function - 需要获取参数名的函数对象。
    
    返回值:
    如果函数参数有效，则返回一个包含所有参数名的列表；否则返回None。
    """
    if function is not None and inspect.isfunction(function):
        # 如果传入的对象是函数，获取其参数名
        parameter_names = inspect.signature(function).parameters.keys()
        return list(parameter_names)
    else:
        # 如果传入的对象不是函数，返回None
        return None

In [17]:
# 获取函数名称及其参数
# 从ai_response_message中提取函数调用的信息，包括函数名和参数
function_name = ai_response_message.function_call.name
arguments = json.loads(ai_response_message.function_call.arguments)

# 定位函数并调用
# 通过函数名在全局变量中查找函数，然后获取函数的参数名，并根据参数名提取参数值，最后调用函数
the_function = globals().get(function_name)  # 尝试从全局变量中获取函数对象
parameter_names = get_function_parameter_names(the_function)  # 获取函数的参数名列表
parameter_values = []
for parameter_name in parameter_names:
  parameter_values.append(arguments[parameter_name])  # 根据参数名提取参数值

returned_value = the_function(*parameter_values)  # 调用函数，并传入参数值列表
returned_value  # 返回函数执行结果


'{"name": "Lucy", "score": 60}'

In [18]:
second_response = client.chat.completions.create(model=model,
                                                 messages=[
                                                     {"role": "user", "content": user_query},
                                                     ai_response_message,
                                                     {
                                                         "role": "function",
                                                         "name": "get_student_score",
                                                         "content": returned_value,
                                                     },
                                                 ])
print (second_response.choices[0].message.content)

Lucy has achieved a score of 60 this year.
